# Projeto Previsão de Churn Rate

### A pergunta inicial e se existe uma correlação entre as variaveis do data-set e com o qual precisão o modelo pode prever a porbabilidade de um cliente evadir

### Variaveis
- Churn: Yes or Not 
- Contract: Month-to-month , One year, Two year
- Dependents: Yes or Not
- DeviceProtection: Yes or Not
- InternetService: Fiber Optic, DSL, No 
- MonthlyCharges= How much the customer spent per month
- MultipleLines: Yes or Not
- OnlineBackup: Yes or Not 
- OnlineSecurity: Yes or Not
- PaperlessBilling: Yes or Not
- Partner: Yes or Not
- PaymentMethod: Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)
- PhoneService: Yes or Not 
- SeniorCitizen: 0 or 1
- StreamingMovies: Yes or Not
- StreamingTV: Yes or Not
- TechSupport: Yes or Not
- TotalCharges= how much the customer spent in total
- customerID: ID
- gender: The customer is a male or a female
- tenure = Months The customer has stayed with the company

Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

Chamando arquivo

In [2]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

Vendo tamanho do arquivo e informações do arquivo

In [3]:
df.shape

(7043, 21)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Convertendo total charges em numerico e encontrando se há valores missing

In [5]:
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

Entendendo os valores Missing

In [9]:
df[['tenure','MonthlyCharges','TotalCharges']].head()

,tenure,MonthlyCharges,TotalCharges
0,1,29.85,29.85
1,34,56.95,1889.50
2,2,53.85,108.15
3,45,42.30,1840.75
4,2,70.70,151.65


Sabemos que os valores de TotalCharges = tenure x MontlyCharges

Sendo assim, Vamos investigar os valores para podermos entender melhor

In [29]:
df['TotalCharges'].sort_values().tail(11)

488    NaN
753    NaN
936    NaN
1082   NaN
1340   NaN
3331   NaN
3826   NaN
4380   NaN
5218   NaN
6670   NaN
6754   NaN
Name: TotalCharges, dtype: float64

In [27]:
df['tenure'].sort_values().head(11)

1340    0
936     0
6670    0
4380    0
5218    0
3826    0
753     0
6754    0
3331    0
488     0
1082    0
Name: tenure, dtype: int64

Podemos notar que todos os valores zerados em tenure tem o mesmo Id dos NaN de TotalCharges

In [11]:
df.query('tenure=="0"').head(12)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


Podemos notar que os valores Missing são clientes ativos com menos de um mês de uso, logo nesse caso o total charges e igual ao montly charges.

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


In [17]:
df.loc[df['StreamingMovies']=='No internet service', 'StreamingMovies'] = 'No'
df.loc[df['StreamingTV']=='No internet service', 'StreamingTV'] = 'No'
df.loc[df['OnlineSecurity']=='No internet service', 'OnlineSecurity'] = 'No'
df.loc[df['OnlineBackup']=='No internet service', 'OnlineBackup'] = 'No'
df.loc[df['DeviceProtection']=='No internet service', 'DeviceProtection'] = 'No'
df.loc[df['TechSupport']=='No internet service', 'TechSupport'] = 'No'
df.loc[df['OnlineSecurity']=='No internet service', 'OnlineSecurity'] = 'No'
df.loc[df['MultipleLines']=='No phone service', 'MultipleLines'] = 'No'

In [18]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [19]:
transform = OrdinalEncoder()

Agora vamos tranformas os Yes ou Not em argumentos binarios 1 ou 0

Não esquecer e especificar a colina após o DF que você quer modificar!

In [20]:
col_bin = ['Churn','gender', 'Partner','Dependents','PhoneService','StreamingTV','PaperlessBilling','StreamingMovies','OnlineSecurity',
           'OnlineBackup', 'DeviceProtection', 'TechSupport', 'OnlineSecurity', 'MultipleLines']
df[col_bin] = pd.DataFrame(pd.DataFrame(columns=col_bin, data=transform.fit_transform(df[col_bin])))

In [21]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0.0,0,1.0,0.0,1,0.0,0.0,DSL,0.0,...,0.0,0.0,0.0,0.0,Month-to-month,1.0,Electronic check,29.85,29.85,0.0
1,5575-GNVDE,1.0,0,0.0,0.0,34,1.0,0.0,DSL,1.0,...,1.0,0.0,0.0,0.0,One year,0.0,Mailed check,56.95,1889.5,0.0
2,3668-QPYBK,1.0,0,0.0,0.0,2,1.0,0.0,DSL,1.0,...,0.0,0.0,0.0,0.0,Month-to-month,1.0,Mailed check,53.85,108.15,1.0
3,7795-CFOCW,1.0,0,0.0,0.0,45,0.0,0.0,DSL,1.0,...,1.0,1.0,0.0,0.0,One year,0.0,Bank transfer (automatic),42.30,1840.75,0.0
4,9237-HQITU,0.0,0,0.0,0.0,2,1.0,0.0,Fiber optic,0.0,...,0.0,0.0,0.0,0.0,Month-to-month,1.0,Electronic check,70.70,151.65,1.0


Agora tranformamos as colunas categoricas menos customerID em colunas

In [22]:
Cat = ['InternetService', 'Contract', 'PaymentMethod']
df = pd.get_dummies(df, columns=Cat)

In [23]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,0.0,0,1.0,0.0,1,0.0,0.0,0.0,1.0,...,1,0,0,1,0,0,0,0,1,0
1,5575-GNVDE,1.0,0,0.0,0.0,34,1.0,0.0,1.0,0.0,...,1,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,1.0,0,0.0,0.0,2,1.0,0.0,1.0,1.0,...,1,0,0,1,0,0,0,0,0,1
3,7795-CFOCW,1.0,0,0.0,0.0,45,0.0,0.0,1.0,0.0,...,1,0,0,0,1,0,1,0,0,0
4,9237-HQITU,0.0,0,0.0,0.0,2,1.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,1,0


## Vamos iniciar a analise exploratoria dos dados

In [24]:
corr_matrix = df.corr()

In [25]:
corr_matrix["Churn"].sort_values(ascending=False)

Churn                                      1.000000
Contract_Month-to-month                    0.405103
InternetService_Fiber optic                0.308020
PaymentMethod_Electronic check             0.301919
MonthlyCharges                             0.193356
PaperlessBilling                           0.191825
SeniorCitizen                              0.150889
StreamingTV                                0.063228
StreamingMovies                            0.061382
MultipleLines                              0.040102
PhoneService                               0.011942
gender                                    -0.008612
DeviceProtection                          -0.066160
OnlineBackup                              -0.082255
PaymentMethod_Mailed check                -0.091683
PaymentMethod_Bank transfer (automatic)   -0.117937
InternetService_DSL                       -0.124214
PaymentMethod_Credit card (automatic)     -0.134302
Partner                                   -0.150448
Dependents  